In [6]:
import sys
sys.path.append("..")

from galpy.potential import evaluatePotentials, MiyamotoNagaiPotential, NFWPotential, HernquistPotential

import numpy as np
from scipy import stats
from IPython.display import Image, display

import corner

import datetime
import random

import copy

import struct

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams

import datetime
%matplotlib inline

rcParams['font.size'] = 16
rcParams['lines.linewidth'] = 1.5
rcParams['axes.linewidth'] = 1.5
rcParams['ytick.major.width'] = 1.5
rcParams['xtick.major.width'] = 1.5
rcParams['image.cmap'] = 'magma'
rcParams['savefig.bbox'] = 'tight'
rcParams['savefig.dpi'] = 300
rcParams['savefig.format'] = 'png'

cmap = copy.copy(plt.cm.get_cmap(rcParams['image.cmap']))
cmap.set_bad((0,0,0))
%config InlineBackend.figure_format = 'retina'


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
machine = 'mac'
#machine = 'magny'
machine = 'virgo'

if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    filedir = "/home/extmilan/masterthesis/files/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
elif machine == 'virgo': 
    basedir = "/virgo/simulations/Auriga/level4_MHD/"
    filedir = "/u/milas/masterthesis/masterproject/files/"
    plotdir = "/u/milas/masterthesis/masterproject/plots/"
   

In [60]:
# Load ages of mergers
age = np.loadtxt(filedir + 'auriga/survivor_overview_all_snapshots.txt',usecols = 1, skiprows=2)

mergertime1 = age[-(128-108)]
mergertime2 = age[-(128-73)]
mergertime3 = age[-(128-68)]    

print('Mergertime of prog2: {}'.format(mergertime1))
print('Mergertime of prog3: {}'.format(mergertime2))
print('Mergertime of prog4: {}'.format(mergertime3))

Mergertime of prog2: 3.15124007648
Mergertime of prog3: 8.69902115816
Mergertime of prog4: 9.46472439035


In [7]:
# Load merger tree info
filename = '/virgo/simulations/Auriga/level4_MHD/lists/halo_24starID_accreted_all_newmtree.dat'

fin = open(filename, "rb")
nacc = struct.unpack( 'i', fin.read(4) )[0]
idacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
subacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
snapacc = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
aflag = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)))
fofflag = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
rootid = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
pkmassid = np.array(struct.unpack('%sd' % nacc, fin.read(nacc*8)), dtype=np.int64)
fin.close()

- aflag  can be (0,1) for accreted stars, or those in an existing subhalo
- rootid is the unique progenitor ID of the subhalo in which the star was born
- pkmassid is the unique progenitor ID of the subhalo that a particle has been in when it was at it's maximum mass before it merged
- idacc is the particle ID
- subacc is the subfind ID of the subhalo is was born in
- snapacc is the snapshot is was born in


In [8]:
def return_pkmassprogid_mostmassive(pkmassid, n=10):
    first_prog = np.array(list(sorted(set(list(pkmassid)))))
    stars_in_subhalo = np.zeros(len(first_prog))
    for i, pid in zip(range(len(first_prog)),first_prog):
        stars_in_subhalo[i] = sum( pkmassid==pid )
    nsort = np.argsort( stars_in_subhalo )[::-1]
    stars_in_subhalo = stars_in_subhalo[nsort].astype('int')
    first_prog = first_prog[nsort]

    return first_prog[:n]

In [9]:
first_prog = return_pkmassprogid_mostmassive(pkmassid)
#pindy, = np.where( (pkmassid==first_prog[0]) )


In [13]:
saved = True
numbers = []
findinsnap = []
ids = []
for i in range(10):
    pindy, = np.where( (pkmassid==first_prog[i]) ) 
    ids.append(idacc[pindy])
    
    numbers.append(len(ids))
    rooty = rootid[pindy]
    snapfrom = snapacc[pindy]
    accretion = aflag[pindy]
    dir_prog = pkmassid[pindy]
    header = 'ID\troot halo\tsnapshot it was born in\taccreted\tlast progenitor'
    #findinsnap.append(np.isin(s.id,ids))
    if saved == False:
        np.savetxt(filedir + '/merger_trees/progenitor{}_info'.format(i+1), ids, fmt='%d')

In [12]:
#look at evolution of ids in which subhalos they are (0th subhalo means main galaxy)

[97553, 84656, 62832, 41581, 16792, 12826, 11098, 5868, 5217, 4263]

In [61]:
level = 4
halo_number = 24

halodir = basedir+"halo_{0}/".format(halo_number)
snappath = halodir+"output/"

startnr = 60
endnr = 128

for snapnr in range(startnr,endnr,1):
    print("level   : {0}".format(level))
    print("halo    : {0}".format(halo_number))
    print("snapnr  : {0}".format(snapnr))
    print("basedir : {0}".format(basedir))
    print("halodir : {0}".format(halodir))
    print("snappath: {0}\n".format(snappath))
    s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=True) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )


    
    for iprog in range(len(ids)):
        i_id_prog = np.isin(s.id, ids[iprog])
        saved_ids = s.id[i_id_prog]
        halonum = s.halo[i_id_prog]
        subhalonum = s.subhalo[i_id_prog]
        mass = s.mass[i_id_prog]
        #print('In snap {} the progenitor {} are in this halo {} and these subhalos{}'.format(snapnr, iprog+1, halonum, subhalonum))
        mainhalo = np.bincount(halonum[np.isfinite(halonum) * (halonum>=0)]).argmax()
        mainsubhalo = np.bincount(subhalonum[np.isfinite(subhalonum)* (subhalonum>=0)]).argmax()
        print('In snap {}, most particles of progenitor {} are in halo {} and in the/its subhalo {}.'.format(snapnr, iprog+1, mainhalo, mainsubhalo))
        savedata = [saved_ids, halonum, subhalonum, mass]
        header = 'ID\thalonum\tsubhalonum\tmass'
        np.savetxt(filedir + 'auriga/subhalo_numbers/subhalo_nums_snap_{}_prog{}'.format(snapnr,iprog+1), savedata, header = header)

level   : 4
halo    : 24
snapnr  : 60
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 84249 stars.
Rotated pos.
Rotated vel.

galrad  : 0.008027402311563491
redshift: 2.1032696525957713
time    : 0.32224076923625916
center  : [ 0.  0.  0.]

In snap 60, most particles of progenitor 1 are in halo 14 and in the/its subhalo 0.
In snap 60, most particles of progenitor 2 are in halo 5 and in the/its subhalo 0.
In snap 60, most particles of progenitor 3 are in halo 2 and in the/its subhalo 0.
In snap 60, most particles of progenitor 4 are in halo 0 and in the/its subhalo 1.
In snap 60, most particles of progenitor 5 are in halo 0 and in the/its subhalo 2.
In snap 60, most particles of progenitor 6 are in halo 0 and in the/its subhalo 3.
In snap 60, most particles of progenitor 7 are in halo 12 and in the/its subhalo 0.
In snap 60, most particles of progenitor 8 are in 

In snap 66, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 66, most particles of progenitor 10 are in halo 0 and in the/its subhalo 3.
level   : 4
halo    : 24
snapnr  : 67
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 371337 stars.
Rotated pos.
Rotated vel.

galrad  : 0.011243060231208801
redshift: 1.5312390291576135
time    : 0.395063440663206
center  : [ 0.  0.  0.]

In snap 67, most particles of progenitor 1 are in halo 6 and in the/its subhalo 0.
In snap 67, most particles of progenitor 2 are in halo 5 and in the/its subhalo 0.
In snap 67, most particles of progenitor 3 are in halo 0 and in the/its subhalo 1.
In snap 67, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 67, most particles of progenitor 5 are in halo 0 and in the/its subhalo 2.
In snap 67, most particles of progenitor 6 are in ha

In snap 73, most particles of progenitor 7 are in halo 0 and in the/its subhalo 3.
In snap 73, most particles of progenitor 8 are in halo 0 and in the/its subhalo 4.
In snap 73, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 73, most particles of progenitor 10 are in halo 0 and in the/its subhalo 0.
level   : 4
halo    : 24
snapnr  : 74
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 754347 stars.
Rotated pos.
Rotated vel.

galrad  : 0.014235840737819673
redshift: 1.1546027123602154
time    : 0.46412268687092223
center  : [ 0.  0.  0.]

In snap 74, most particles of progenitor 1 are in halo 5 and in the/its subhalo 0.
In snap 74, most particles of progenitor 2 are in halo 0 and in the/its subhalo 1.
In snap 74, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 74, most particles of progenitor 4 are in 

In snap 80, most particles of progenitor 5 are in halo 0 and in the/its subhalo 2.
In snap 80, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 80, most particles of progenitor 7 are in halo 0 and in the/its subhalo 6.
In snap 80, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 80, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 80, most particles of progenitor 10 are in halo 0 and in the/its subhalo 0.
level   : 4
halo    : 24
snapnr  : 81
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 925630 stars.
Rotated pos.
Rotated vel.

galrad  : 0.016126376390457154
redshift: 0.8868969375752482
time    : 0.5299706518603222
center  : [ 0.  0.  0.]

In snap 81, most particles of progenitor 1 are in halo 5 and in the/its subhalo 0.
In snap 81, most particles of progenitor 2 are in h

In snap 87, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 87, most particles of progenitor 10 are in halo 0 and in the/its subhalo 0.
level   : 4
halo    : 24
snapnr  : 88
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 1154417 stars.
Rotated pos.
Rotated vel.

galrad  : 0.017521795630455018
redshift: 0.6761104112134777
time    : 0.596619407

In snap 94, most particles of progenitor 1 are in halo 0 and in the/its subhalo 1.
In snap 94, most particles of progenitor 2 are in halo 0 and in the/its subhalo 2.
In snap 94, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 94, most particles of progenitor 10 are in halo 0 and in the/its subhalo 0.
level   : 4
halo    : 24
snapnr  : 95
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulation

Found 1484340 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02092690169811249
redshift: 0.38016786726023866
time    : 0.7245495448210172
center  : [ 0.  0.  0.]

In snap 101, most particles of progenitor 1 are in halo 0 and in the/its subhalo 1.
In snap 101, most particles of progenitor 2 are in halo 0 and in the/its subhalo 3.
In snap 101, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 101, most particles of progenitor 10 are in halo 0 and in the/its subhalo 

Found 1625123 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02170633673667908
redshift: 0.2613432561610123
time    : 0.7928056023731169
center  : [ 0.  0.  0.]

In snap 108, most particles of progenitor 1 are in halo 0 and in the/its subhalo 1.
In snap 108, most particles of progenitor 2 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 108, most particles of progenitor 10 are in halo 0 and in the/its subhalo 0

Found 1689680 stars.
Rotated pos.
Rotated vel.

galrad  : 0.023059283196926118
redshift: 0.15274876890238098
time    : 0.8674917093619414
center  : [ 0.  0.  0.]

In snap 115, most particles of progenitor 1 are in halo 0 and in the/its subhalo 1.
In snap 115, most particles of progenitor 2 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 115, most particles of progenitor 10 are in halo 0 and in the/its subhalo

Found 1755643 stars.
Rotated pos.
Rotated vel.

galrad  : 0.023272740840911868
redshift: 0.058507322794512984
time    : 0.9447265771954694
center  : [ 0.  0.  0.]

In snap 122, most particles of progenitor 1 are in halo 0 and in the/its subhalo 1.
In snap 122, most particles of progenitor 2 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 3 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 4 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 5 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 6 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 7 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 8 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 9 are in halo 0 and in the/its subhalo 0.
In snap 122, most particles of progenitor 10 are in halo 0 and in the/its subhal

In [68]:
level = 4
halo_number = 24

halodir = basedir+"halo_{0}/".format(halo_number)
snappath = halodir+"output/"

startnr = 60
endnr = 128

#63: [0,4]
#72: [0,1]
#107: [0,4]
for snapnr in [63,72,104, 105, 106, 107]:
    print("level   : {0}".format(level))
    print("halo    : {0}".format(halo_number))
    print("snapnr  : {0}".format(snapnr))
    print("basedir : {0}".format(basedir))
    print("halodir : {0}".format(halodir))
    print("snappath: {0}\n".format(snappath))
    s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=True) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )

    if snapnr == 63:
        iprog = 3
        
    elif snapnr == 72:
        iprog = 2
        
    elif np.any(np.isin([104,105,106,107], snapnr)):
        iprog = 1
    
    print(iprog)
    
    i_id_prog = np.isin(s.id, ids[iprog])
        #saved_ids = s.id[i_id_prog]
    halonum = s.halo[i_id_prog]
    subhalonum = s.subhalo[i_id_prog]
        #print('In snap {} the progenitor {} are in this halo {} and these subhalos{}'.format(snapnr, iprog+1, halonum, subhalonum))
    mainhalo = np.bincount(halonum[np.isfinite(halonum) * (halonum>=0)]).argmax()
    mainsubhalo = np.bincount(subhalonum[np.isfinite(subhalonum)* (subhalonum>=0)]).argmax()
    print('In snap {}, most particles of progenitor {} are in halo {} and in the/its subhalo {}.'.format(snapnr, iprog+1, mainhalo, mainsubhalo))
    
    mergers = (halonum == mainhalo) * (subhalonum == mainsubhalo)
    mass = np.sum(s.mass[i_id_prog][mergers])
    print('Progenitor {} has {} particles and a mass of {} 10^10 M_sun in the snap before the merger.'.format(iprog+1, np.sum(mergers),mass) )
    
    DG = (s.halo == mainhalo) * (s.subhalo == mainsubhalo)
    masscompare = np.sum(s.mass[DG])
    print('Dwarf galaxy [{},{}] has {} particles and a mass of {} 10^10 M_sun in the snap before the merger.'.format(mainhalo, mainsubhalo, np.sum(DG),masscompare) )
    

level   : 4
halo    : 24
snapnr  : 63
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 164288 stars.
Rotated pos.
Rotated vel.

galrad  : 0.009642120450735092
redshift: 1.822689252620354
time    : 0.3542720825792927
center  : [ 0.  0.  0.]

3
In snap 63, most particles of progenitor 4 are in halo 0 and in the/its subhalo 4.
Progenitor 4 has 31826 particles and a mass of 0.13585320115089417 10^10 M_sun in the snap before the merger.
Dwarf galaxy [0,4] has 32404 particles and a mass of 0.13896223902702332 10^10 M_sun in the snap before the merger.
level   : 4
halo    : 24
snapnr  : 72
basedir : /virgo/simulations/Auriga/level4_MHD/
halodir : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath: /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 536367 stars.
Rotated pos.
Rotated vel.

galrad  : 0.013163433969020845
redshift: 1.2484726142451428
time    : 

In [37]:
len(saveids[4])

6204

In [33]:
len(subhalos[4])

6204

In [51]:
np.bincount(subhalonum).argmax()

0

In [48]:
np.min(subhalonum)

0

In [49]:
np.max(subhalonum).a

12

In [50]:
np.median(subhalonum)

0.0